In [ ]:
# | default_exp classes.DomoDataflow_Action

In [ ]:
# | exporti

from __future__ import annotations

from enum import Enum
from dataclasses import dataclass
from typing import List

import datetime as dt
import domolibrary.utils.DictDot as util_dd
import domolibrary.utils.convert as ct

from nbdev.showdoc import  patch_to

In [ ]:
#|hide
import domolibrary.routes.dataflow as dataflow_routes
import domolibrary.client.DomoAuth as dmda

from nbdev.showdoc import show_doc

In [ ]:
# | export

class DomoDataflow_Action_Type(Enum):
    LoadFromVault = "LoadFromVault"
    PublishToVault = "PublishToVault"
    GenerateTableAction = "GenerateTableAction"

@dataclass 
class DomoAction:
    id: str
    type: str = None
    name: str = None

@dataclass
class DomoDataflow_Action(DomoAction):
    datasource_id: str  = None
    sql: str = None

    depends_on : List[str] = None
    parent_actions : List[dict] = None


    @classmethod
    def _from_json(cls, obj: dict):
        dd = obj

        if isinstance(dd, dict):
            dd = util_dd.DictDot(obj)

        tbl_name = dd.dataSource.name if dd.dataSource else None
        ds_id = dd.dataSource.guid if dd.dataSource else None

        return  cls(
            type=dd.type,
            id=dd.id,
            name=dd.name or dd.targetTableName or dd.tableName or tbl_name,
            depends_on = dd.dependsOn,
            
            datasource_id=dd.dataSourceId or ds_id,
            sql=dd.selectStatement or dd.query,
        )



In [ ]:
#| exporti

@patch_to(DomoDataflow_Action)
def get_parents(self: DomoDataflow_Action, domo_actions: List[DomoDataflow_Action]):
    if self.depends_on and len(self.depends_on) > 0:
        self.parent_actions = [
            parent_action
            for depends_id in self.depends_on
            for parent_action in domo_actions
            if parent_action.id == depends_id
        ]
    
    if self.parent_actions:
        [parent.get_parents(domo_actions ) for parent in self.parent_actions if parent.depends_on]
            

    return self.parent_actions

#### sample implementation of DataFlow_Action

Note: this list of properties that can be attached to `DomoDataflow_Action` is by no means comprehensive. To capture more data from the API, simply add properties to the class and extend the `DomoDataflow_Action._from_json` private method.


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

dataflow_id = 70
dataflow_id = 185

res = await dataflow_routes.get_dataflow_by_id(dataflow_id=dataflow_id, auth=token_auth)
actions_ls = res.response.get('actions')

[
    DomoDataflow_Action._from_json(action)
    for action in actions_ls
]

[DomoDataflow_Action(id='b9bb0f01-2517-4595-a077-c9778e5ffc48', type='LoadFromVault', name='monit_user_accesslist', datasource_id='6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf', sql=None, depends_on=None, parent_actions=None),
 DomoDataflow_Action(id='52f47029-06e0-4599-9ed8-21c4b9386c89', type='WindowAction', name='Rank & Window', datasource_id=None, sql=None, depends_on=['b9bb0f01-2517-4595-a077-c9778e5ffc48'], parent_actions=None),
 DomoDataflow_Action(id='623883e9-bdc1-44cd-9fb4-6277d0e123de', type='ExpressionEvaluator', name='Add Formula', datasource_id=None, sql=None, depends_on=['52f47029-06e0-4599-9ed8-21c4b9386c89'], parent_actions=None),
 DomoDataflow_Action(id='99ad5978-9a19-41e1-99d7-3d0ce58f7467', type='Metadata', name='Alter Columns', datasource_id=None, sql=None, depends_on=['623883e9-bdc1-44cd-9fb4-6277d0e123de'], parent_actions=None),
 DomoDataflow_Action(id='4cf42103-c52d-4b92-b26f-63832ead11ee', type='SelectValues', name='Select Columns', datasource_id=None, sql=None, depend

In [ ]:
#| export

@dataclass
class DomoDataflow_ActionResult(DomoAction):
    is_success : bool = None
    rows_processed : int = None
    begin_time : dt.datetime = None
    end_time : dt.datetime = None
    duration_in_sec : int  = None

    def __post_init__(self):
        if self.begin_time and self.end_time:
            self.duration_in_sec =  (self.end_time - self.begin_time).total_seconds()
            
    @classmethod
    def _from_json(cls, obj: dict):
        return  cls(
            id=obj.get('actionId'),
            type = obj.get('type'),
            is_success = obj.get('wasSuccessful'),
            begin_time = ct.convert_epoch_millisecond_to_datetime(obj.get('beginTime', None)),
            end_time = ct.convert_epoch_millisecond_to_datetime(obj.get('endTime', None)),
            rows_processed =  obj.get('rowsProcessed', None),
        )


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await dataflow_routes.get_dataflow_execution_by_id(
    dataflow_id=70, execution_id=433238, auth=token_auth
)

action_ls = res.response['actionResults']

[DomoDataflow_ActionResult._from_json(action_obj) for action_obj in action_ls]

[DomoDataflow_ActionResult(id='8edc3d78-3586-47a8-9775-c7b146a50d19', type='PublishToVault', name=None, is_success=True, rows_processed=1816, begin_time=datetime.datetime(2023, 12, 5, 6, 17, 1, 487000), end_time=datetime.datetime(2023, 12, 5, 6, 17, 2, 754000), duration_in_sec=1.267),
 DomoDataflow_ActionResult(id='c176aa53-122a-4a2e-8b6f-41ca4a72d67c', type='GenerateTableAction', name=None, is_success=True, rows_processed=909, begin_time=datetime.datetime(2023, 12, 5, 6, 17, 0, 983000), end_time=datetime.datetime(2023, 12, 5, 6, 17, 1, 100000), duration_in_sec=0.117),
 DomoDataflow_ActionResult(id='241025d7-3cca-4369-b7c0-b3264277c0e1', type='DataHubManifestLoaderAction', name=None, is_success=True, rows_processed=909, begin_time=datetime.datetime(2023, 12, 5, 6, 16, 55, 894000), end_time=datetime.datetime(2023, 12, 5, 6, 17, 0, 972000), duration_in_sec=5.078),
 DomoDataflow_ActionResult(id='ac9f630d-d865-4925-b955-4316e227229a', type='GenerateTableAction', name=None, is_success=True,

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()